In [1]:
import keras
import numpy as np

import pynucastro as pyna

import chugunov_indicator as chug

fixed_rng = np.random.default_rng(0)

We have reduced our original task to a binary classification problem of the form
$$
    s = s(y_0; \bar{A}, \overline{Z^2}; Z_1, Z_2) \in \{0, 1\}, \quad y_0 := 3 \log_{10} T - \log_{10} D.
$$
To solve this problem without manual fitting, we could also try training a neural network in `keras` that predicts when screening can be skipped.

In [2]:
kwargs = {"size": 3*10**5, "rng": fixed_rng}

train = chug.ScreeningFactorData(**kwargs)
validate = chug.ScreeningFactorData(**kwargs)
test = chug.ScreeningFactorData(**kwargs)

network = chug.ScreeningFactorNetwork(train, validate, test, 0)

In [3]:
network.class_weight = {0: 64983, 1: 35017*2}
network.compile(learning_rate=1e-4)

In [4]:
history = network.fit_model(epochs=500, verbose=2)
network.score

Epoch 1/500
147/147 - 4s - 29ms/step - fn: 16226.0000 - fp: 14267.0000 - loss: 23067.8379 - precision: 0.8612 - recall: 0.8451 - tn: 181004.0000 - tp: 88503.0000 - val_fn: 4684.0000 - val_fp: 6376.0000 - val_loss: 0.1046 - val_precision: 0.9402 - val_recall: 0.9554 - val_tn: 188621.0000 - val_tp: 100319.0000
Epoch 2/500
147/147 - 2s - 11ms/step - fn: 3893.0000 - fp: 6103.0000 - loss: 5712.7471 - precision: 0.9429 - recall: 0.9628 - tn: 189168.0000 - tp: 100836.0000 - val_fn: 3215.0000 - val_fp: 4057.0000 - val_loss: 0.0613 - val_precision: 0.9617 - val_recall: 0.9694 - val_tn: 190940.0000 - val_tp: 101788.0000
Epoch 3/500
147/147 - 2s - 11ms/step - fn: 2689.0000 - fp: 4063.0000 - loss: 3789.4741 - precision: 0.9617 - recall: 0.9743 - tn: 191208.0000 - tp: 102040.0000 - val_fn: 2271.0000 - val_fp: 2642.0000 - val_loss: 0.0434 - val_precision: 0.9749 - val_recall: 0.9784 - val_tn: 192355.0000 - val_tp: 102732.0000
Epoch 4/500
147/147 - 2s - 12ms/step - fn: 2172.0000 - fp: 2921.0000 - los

[0.0038521611131727695,
 303.0,
 161.0,
 0.9984671473503113,
 0.9971190690994263,
 194665.0,
 104871.0]

In [3]:
#network.model.save("chugunov_indicator/keras/model.keras")
network.model = keras.saving.load_model("chugunov_indicator/keras/model.keras")

In [4]:
nuclei = pyna.Nucleus.cast_list(["h1", "he4", "c12", "o16", "n14", "ca40"])
reactants = pyna.Nucleus.cast_list(["c12", "he4"])

comp = pyna.Composition(nuclei=nuclei)
comp.set_solar_like()

scn_fac = pyna.make_screen_factors(*reactants)

In [5]:
dens, temp = 1.e6, 2e8
plasma = pyna.make_plasma_state(temp, dens, comp.get_molar())

In [6]:
pyna.screening.chugunov_2009(plasma, scn_fac)

1.7457134746043816

In [7]:
network.predict_states(plasma, scn_fac)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


False

False negatives are mostly irrelevant, but false positives can cause significant errors to propagate through the system if not caught. Fortunately, an inspection of the false positives within the test data reveals that they all have screening factors within the interval $[1.01, 1.013]$, implying that the neural network is largely successful at its task.

However, the `network.model.predict` method operates on the order of milliseconds, while an analysis `5. speed.ipynb` notebook reveals that a Python implementation of `chugunov_2009` itself operates on the order of microseconds. As such, a neural network like this is would not be effective for optimizing the runtime of `chugunov_2009`.

In [30]:
fp = ((network.model.predict(test.inputs).squeeze() >= 0.5) & ~test.outputs) == 1

9375/9375 ━━━━━━━━━━━━━━━━━━━━ 8s 871us/step


In [31]:
display(
    F_fp := test.F[fp == 1],
    F_fp.size,
    F_fp.max()
)

array([1.01001194, 1.01020107, 1.01000773, 1.01019415, 1.01000966,
       1.01030038, 1.0103661 , 1.0104646 , 1.01054024, 1.01024759,
       1.01068491, 1.01040766, 1.01085853, 1.01008068, 1.01079409,
       1.01020378, 1.01002502, 1.01024663, 1.01000194, 1.0102119 ,
       1.01004076, 1.0101782 , 1.01029973, 1.01016429, 1.0105595 ,
       1.01005702, 1.01003274, 1.01213755, 1.01001769, 1.01009793,
       1.01016006, 1.01057366, 1.01002678, 1.01044191, 1.01127694,
       1.01010192, 1.01017348, 1.01089798, 1.01000898, 1.01031632,
       1.01055381, 1.01056098, 1.01180212, 1.01005668, 1.01007959,
       1.01050641, 1.01034203, 1.01013163, 1.01003947, 1.010122  ,
       1.01065727, 1.01009619, 1.01025271, 1.01010378, 1.01081283,
       1.01015662, 1.01000324, 1.01015354, 1.0110943 , 1.01010499,
       1.01007202, 1.01000768, 1.01035353, 1.01020641, 1.01002173,
       1.01026171, 1.01107069, 1.01095777, 1.01004197, 1.01022607,
       1.01009426, 1.01099453, 1.010289  , 1.01051833, 1.01064

161

1.0129651196810776